# Segmenting Toronto Neighborhoods with K-Means Clustering

First we have to input the necessary libraries

In [43]:
import pandas as pd
import numpy as np
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
import folium
import geocoder
from sklearn.cluster import DBSCAN 
print('Libraries imported.')

Libraries imported.


In [2]:
print('hello')

hello


Check status code to make sure we can scrape the website

In [4]:
wikiurl = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
print(response.status_code)

200


Use Beautiful Soup to get the html of the dataframe

In [5]:
soup = BeautifulSoup(response.text, 'html.parser')
torontoHTML=soup.find('table',{'class':"wikitable"})

Read the html into a pandas dataframe and drop rows where the Borough is not assigned

In [6]:
torontoTable =pd.DataFrame(pd.read_html(str(torontoHTML))[0])
torontoTable.drop(torontoTable[torontoTable['Borough'] == 'Not assigned'].index, axis = 0, inplace = True)

Check if there are any rows where the neighbourhood is not assigned or any equivalent

In [7]:
torontoTable['Neighbourhood'].unique()

array(['Parkwoods', 'Victoria Village', 'Regent Park, Harbourfront',
       'Lawrence Manor, Lawrence Heights',
       "Queen's Park, Ontario Provincial Government",
       'Islington Avenue, Humber Valley Village', 'Malvern, Rouge',
       'Don Mills', 'Parkview Hill, Woodbine Gardens',
       'Garden District, Ryerson', 'Glencairn',
       'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale',
       'Rouge Hill, Port Union, Highland Creek', 'Woodbine Heights',
       'St. James Town', 'Humewood-Cedarvale',
       'Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood',
       'Guildwood, Morningside, West Hill', 'The Beaches', 'Berczy Park',
       'Caledonia-Fairbanks', 'Woburn', 'Leaside', 'Central Bay Street',
       'Christie', 'Cedarbrae', 'Hillcrest Village',
       'Bathurst Manor, Wilson Heights, Downsview North',
       'Thorncliffe Park', 'Richmond, Adelaide, King',
       'Dufferin, Dovercourt Village', 'Scarborough Village',
       'Fairview, H

It doesn't seem like there are any. Now we show the DataFrame

In [8]:
torontoTable.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Find the shape of the DataFrame

In [9]:
torontoTable.shape

(103, 3)

Let's see if geocoder will work. We will run 50 iterations and if that doesn't work we'll try something else.

In [19]:
i = 0
postal_code = 'M3A'
lat_lng_coords = None
while(lat_lng_coords is None):
    if i<50:
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        i+=1
        print(str(i))
    else:
        print('50 iterations and still no coordinates')
        break

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
50 iterations and still no coordinates


TypeError: 'NoneType' object is not subscriptable

I don't think it will, so let's read the csv provided and set the index to the Postal Code to make joining the Dataframes easier

In [20]:
coordinates_df = pd.read_csv('Geospatial_Coordinates (1).csv')
coordinates_df.set_index('Postal Code', inplace = True)

Now let's join the two dataframes and see the result

In [21]:
torontoTable = torontoTable.join(coordinates_df, on = 'Postal Code', how = 'left')

In [22]:
torontoTable.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Let's rename the column neighbourhood to neighborhood to make it easier for us later

In [106]:
torontoTable.rename(columns = {'Neighbourhood':'Neighborhood'}, inplace = True)

I want to cluster the neighbourhoods based off of their food related venues. First I have to define all the constants within my url.

In [23]:
CLIENT_ID = 'Z2BADHUK0WDYPWNCJKJ11VB5EREKVKIKZMD5JR5NO4IB2QV3'
CLIENT_SECRET = 'IZ1T1SGBG3HPOHRG4JE20IDOV1ITJFTCBVLEMAGC1D1HVUDF'
VERSION = '20180604'
radius = 1500
LIMIT = 20
search_query = 'food'

In [24]:
lat = 43.753259
lng = -79.329656
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, search_query, radius, LIMIT)
testresults = requests.get(url).json()['response']['venues']
testresults[7]


{'id': '4b4f703ff964a520990627e3',
 'name': 'Luxmy foods',
 'location': {'lat': 43.74141,
  'lng': -79.314378,
  'labeledLatLngs': [{'label': 'display', 'lat': 43.74141, 'lng': -79.314378}],
  'distance': 1802,
  'cc': 'CA',
  'country': 'Canada',
  'formattedAddress': ['Canada']},
 'categories': [],
 'referralId': 'v-1606399043',
 'hasPerk': False}

In [96]:
for i in range(10):
    print(str(i))
results[9]['categories'][0]['name']
[item for venue_list in venues_list for item in venue_list]

0
1
2
3
4
5
6
7
8
9


Let's make a function to find the nearby food venues

In [28]:
def findCloseFoodVenues(names, latitudes, longitudes, radius=1500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, search_query, radius, LIMIT)
        results = requests.get(url).json()['response']['venues']
        print(name)
        for v in range(len(results)):
            if len(results[v]['categories'])>0:
                category = results[v]['categories'][0]['name']
            else:
                category = 'No Category'
            venues_list.append([
            name, 
            lat, 
            lng, 
            results[v]['name'], 
            results[v]['location']['lat'], 
            results[v]['location']['lng'],
            category])

    nearby_venues = pd.DataFrame(columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category'])
    for row in venues_list:
        to_append = row
        nearby_venues.loc[len(nearby_venues)] = to_append

    
    return(nearby_venues)

In [107]:
torontoTable.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Now let's run the function on each row in the DataFrame and store the results in a new Dataframe

In [108]:
food_venues = findCloseFoodVenues(torontoTable['Neighborhood'], torontoTable['Latitude'], torontoTable['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

Let's look at the new Dataframe

In [109]:
food_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Food Basics,43.760549,-79.326045,Supermarket
1,Parkwoods,43.753259,-79.329656,Family Food Fair Convenience,43.760620,-79.324459,Convenience Store
2,Parkwoods,43.753259,-79.329656,Kraft Foods,43.759407,-79.350730,Office
3,Parkwoods,43.753259,-79.329656,Bruno's Fine Foods,43.745608,-79.336772,Grocery Store
4,Parkwoods,43.753259,-79.329656,Viking Foods,43.739869,-79.320581,Market


How many venues are there for each neighborhood?

In [110]:
food_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,20,20,20,20,20,20
"Alderwood, Long Branch",10,10,10,10,10,10
"Bathurst Manor, Wilson Heights, Downsview North",1,1,1,1,1,1
Bayview Village,1,1,1,1,1,1
"Bedford Park, Lawrence Manor East",16,16,16,16,16,16
...,...,...,...,...,...,...
"Willowdale, Willowdale West",2,2,2,2,2,2
Woburn,6,6,6,6,6,6
Woodbine Heights,12,12,12,12,12,12


How many unique categories are there?

In [111]:
print('There are {} unique categories.'.format(len(food_venues['Venue Category'].unique())))

There are 123 unique categories.


Now let's make a dataframe with numerical values for each category so we can cluster the neighborhoods

In [112]:
food_onehot = pd.get_dummies(food_venues[['Venue Category']], prefix="", prefix_sep="")
food_onehot['Neighborhood'] = food_venues['Neighborhood'] 
fixed_columns = [food_onehot.columns[-1]] + list(food_onehot.columns[:-1])
food_onehot = food_onehot[fixed_columns]
food_onehot.head()

,Neighborhood,Adult Education Center,Afghan Restaurant,African Restaurant,American Restaurant,Arcade,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bakery,...,Tech Startup,Thai Restaurant,Theme Park Ride / Attraction,Theme Restaurant,Tour Provider,Turkish Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Warehouse Store,Weight Loss Center
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Now let's group the columns by neighborhood and average the amount for each category

In [113]:
food_grouped = food_onehot.groupby('Neighborhood').mean().reset_index()
food_grouped

,Neighborhood,Adult Education Center,Afghan Restaurant,African Restaurant,American Restaurant,Arcade,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bakery,...,Tech Startup,Thai Restaurant,Theme Park Ride / Attraction,Theme Restaurant,Tour Provider,Turkish Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Warehouse Store,Weight Loss Center
0,Agincourt,0.0,0.0,0.0,0.0,0.000000,0.0,0.0500,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.000000,0.0,0.0000,0.0,0.0,...,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.000000,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.000000,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.000000,0.0,0.0625,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.000000,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
94,Woburn,0.0,0.0,0.0,0.0,0.000000,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95,Woodbine Heights,0.0,0.0,0.0,0.0,0.000000,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,York Mills West,0.0,0.0,0.0,0.0,0.066667,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now we're going to look at the top five venue categories in each neighborhood. First we make a function to get the top venue categories in a neighborhood

In [114]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now we're going to use this function to give us a dataframe that contains the information we want.

In [116]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = food_grouped['Neighborhood']

for ind in np.arange(food_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(food_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Agincourt,No Category,Food Court,Grocery Store,Supermarket,Gourmet Shop
1,"Alderwood, Long Branch",Food Court,Caribbean Restaurant,Gourmet Shop,Convenience Store,Factory
2,"Bathurst Manor, Wilson Heights, Downsview North",Grocery Store,Distribution Center,Drugstore,Dumpling Restaurant,Eastern European Restaurant
3,Bayview Village,Housing Development,Weight Loss Center,Food,Drugstore,Dumpling Restaurant
4,"Bedford Park, Lawrence Manor East",Grocery Store,Filipino Restaurant,Food & Drink Shop,Food Service,No Category


Now we're going to cluster the neighborhoods with DBSCAN.

In [117]:
food_grouped_clustering = food_grouped.drop('Neighborhood', 1)
db = DBSCAN(eps = 0.25, min_samples = 3).fit(food_grouped_clustering)

Now let's look at what are labels are. First we need to make a function to find unique values in a list

In [142]:
def unique(list1): 
    unique_list = [] 
    for x in list1: 
        if x not in unique_list: 
            unique_list.append(x) 
    return unique_list

Now let's make the labels into a list and look at the unique values

In [143]:
unique(db.labels_.tolist())

[1, -1, 0, 2]

In [120]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Agincourt,No Category,Food Court,Grocery Store,Supermarket,Gourmet Shop
1,"Alderwood, Long Branch",Food Court,Caribbean Restaurant,Gourmet Shop,Convenience Store,Factory
2,"Bathurst Manor, Wilson Heights, Downsview North",Grocery Store,Distribution Center,Drugstore,Dumpling Restaurant,Eastern European Restaurant
3,Bayview Village,Housing Development,Weight Loss Center,Food,Drugstore,Dumpling Restaurant
4,"Bedford Park, Lawrence Manor East",Grocery Store,Filipino Restaurant,Food & Drink Shop,Food Service,No Category


In [121]:
torontoTable.sort_values('Neighborhood')

,Postal Code,Borough,Neighborhood,Latitude,Longitude
117,M1S,Scarborough,Agincourt,43.794200,-79.262029
151,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
47,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259
64,M2K,North York,Bayview Village,43.786947,-79.385975
85,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750
...,...,...,...,...,...
109,M2R,North York,"Willowdale, Willowdale West",43.782736,-79.442259
36,M1G,Scarborough,Woburn,43.770992,-79.216917
21,M4C,East York,Woodbine Heights,43.695344,-79.318389
100,M2P,North York,York Mills West,43.752758,-79.400049


Now let's make a Dataframe that contains the clustering data and the top 5 most popular venue categories

In [124]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', db.labels_)
food_merged = torontoTable
food_merged = food_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
food_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,M3A,North York,Parkwoods,43.753259,-79.329656,-1.0,Grocery Store,Caribbean Restaurant,Supermarket,Office,Convenience Store
3,M4A,North York,Victoria Village,43.725882,-79.315572,-1.0,Grocery Store,Deli / Bodega,Food,No Category,Pub
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0.0,Food & Drink Shop,Food Truck,Health Food Store,College Cafeteria,Café
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,-1.0,Food Court,Café,Food Service,Salad Place,Grocery Store
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0.0,Food Court,Grocery Store,Food Truck,Mediterranean Restaurant,Caribbean Restaurant


Let's look at each cluster and come up with a name for it

In [126]:
food_merged.loc[food_merged['Cluster Labels'] == -1, food_merged.columns[[1] + list(range(5, food_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,North York,-1.0,Grocery Store,Caribbean Restaurant,Supermarket,Office,Convenience Store
3,North York,-1.0,Grocery Store,Deli / Bodega,Food,No Category,Pub
5,North York,-1.0,Food Court,Café,Food Service,Salad Place,Grocery Store
8,Etobicoke,-1.0,Grocery Store,Restaurant,BBQ Joint,Pet Store,Drugstore
9,Scarborough,-1.0,Pet Store,Business Service,Caribbean Restaurant,Indian Restaurant,Farmers Market
11,North York,-1.0,Grocery Store,Food Truck,Middle Eastern Restaurant,Office,Deli / Bodega
12,East York,-1.0,Grocery Store,Cantonese Restaurant,Deli / Bodega,Food,Convenience Store
14,North York,-1.0,Food Court,Grocery Store,No Category,Food & Drink Shop,Building
17,Etobicoke,-1.0,Grocery Store,Pet Store,Office,Chinese Restaurant,Café
18,Scarborough,-1.0,Pet Service,Miscellaneous Shop,Weight Loss Center,Flea Market,Drugstore


In [127]:
food_merged.loc[food_merged['Cluster Labels'] == 0, food_merged.columns[[1] + list(range(5, food_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,Downtown Toronto,0.0,Food & Drink Shop,Food Truck,Health Food Store,College Cafeteria,Café
6,Downtown Toronto,0.0,Food Court,Grocery Store,Food Truck,Mediterranean Restaurant,Caribbean Restaurant
13,Downtown Toronto,0.0,Food Court,Food Truck,Grocery Store,Chinese Restaurant,Mediterranean Restaurant
22,Downtown Toronto,0.0,Food Court,Food Truck,Grocery Store,Salad Place,Fast Food Restaurant
23,York,0.0,Food & Drink Shop,Grocery Store,Chinese Restaurant,Pharmacy,Taco Place
30,East Toronto,0.0,Grocery Store,Chinese Restaurant,Convenience Store,Health Food Store,Pet Store
31,Downtown Toronto,0.0,Food Court,Food Truck,Grocery Store,College Cafeteria,Mediterranean Restaurant
40,Downtown Toronto,0.0,Food Court,Food Truck,Grocery Store,Deli / Bodega,Mediterranean Restaurant
41,Downtown Toronto,0.0,Grocery Store,Convenience Store,Food & Drink Shop,Gourmet Shop,Weight Loss Center
49,Downtown Toronto,0.0,Food Court,Food Truck,Grocery Store,Fast Food Restaurant,Salad Place


In [128]:
food_merged.loc[food_merged['Cluster Labels'] == 1, food_merged.columns[[1] + list(range(5, food_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
57,East York,1.0,Grocery Store,No Category,Fruit & Vegetable Store,Supermarket,Organic Grocery
66,East Toronto,1.0,No Category,Pet Store,Grocery Store,Supermarket,Restaurant
117,Scarborough,1.0,No Category,Food Court,Grocery Store,Supermarket,Gourmet Shop


In [129]:
food_merged.loc[food_merged['Cluster Labels'] == 2, food_merged.columns[[1] + list(range(5, food_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
82,North York,2.0,Food Court,Grocery Store,Candy Store,Housing Development,Indian Restaurant
91,North York,2.0,Food Court,Grocery Store,Pet Store,Eastern European Restaurant,Indian Restaurant
100,North York,2.0,Grocery Store,Butcher,Food Court,Convenience Store,Japanese Restaurant


They all seem to have a lot of food courts. Outside of that, it seems that cluster -1 has a lot of grocery stores, cluster 0 has a lot of food trucks, cluster 1 has mostly no category, and cluster 2 is a mix. Let's rename the Cluster labels.

In [136]:
food_merged['Cluster Labels'].replace(-1, 'Grocery', inplace = True)
food_merged['Cluster Labels'].replace(0, 'Truck', inplace = True)
food_merged['Cluster Labels'].replace(1, 'Unknown', inplace = True)
food_merged['Cluster Labels'].replace(2, 'Misc.', inplace = True)

Now let's add a column with a number 1 through 5 for each cluster

In [169]:
cluster_num = []
for row in food_merged.index:
    if food_merged.loc[row, 'Cluster Labels'] == 'Grocery':
        cluster_num.append(1)
    elif food_merged.loc[row, 'Cluster Labels'] == 'Truck':
        cluster_num.append(2)
    elif food_merged.loc[row, 'Cluster Labels'] == 'Unknown':
        cluster_num.append(3)
    elif food_merged.loc[row, 'Cluster Labels'] == 'Misc.':
        cluster_num.append(4)
    else:
        cluster_num.append(5)

In [171]:
food_merged.insert(0, 'Cluster Numbers', cluster_num)

Finally let's see the dataframe

In [172]:
food_merged.head()

,Cluster Numbers,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,1,M3A,North York,Parkwoods,43.753259,-79.329656,Grocery,Grocery Store,Caribbean Restaurant,Supermarket,Office,Convenience Store
3,1,M4A,North York,Victoria Village,43.725882,-79.315572,Grocery,Grocery Store,Deli / Bodega,Food,No Category,Pub
4,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,Truck,Food & Drink Shop,Food Truck,Health Food Store,College Cafeteria,Café
5,1,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,Grocery,Food Court,Café,Food Service,Salad Place,Grocery Store
6,2,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Truck,Food Court,Grocery Store,Food Truck,Mediterranean Restaurant,Caribbean Restaurant


Now let's visualize the clusters with a folium map

In [175]:
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

x = np.arange(5)
ys = [i + x + (i*x)**2 for i in range(5)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
cluster_numbers = np.arange(1, len(unique(food_merged['Cluster Labels'].tolist())), 1)
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, number in zip(food_merged['Latitude'], food_merged['Longitude'], food_merged['Neighborhood'], food_merged['Cluster Labels'], food_merged['Cluster Numbers']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[number-1],
        fill=True,
        fill_color=rainbow[number-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters